<a href="https://colab.research.google.com/github/gjlr2000/Intelligence-Amplification/blob/master/Kumar_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install mysql-connector-python-rf
! pip install sshtunnel
import mysql.connector
import sshtunnel

sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0



    100% |████████████████████████████████| 11.9MB 951kB/s 
  Running setup.py bdist_wheel for mysql-connector-python-rf ... - \ done
  Stored in directory: /root/.cache/pip/wheels/87/58/fb/d95c84fad7e1bebfed324c13e107ebb08e1997c9226532859a
Successfully built mysql-connector-python-rf
    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 194kB 4.4MB/s 
    100% |████████████████████████████████| 768kB 7.2MB/s 
    100% |████████████████████████████████| 2.1MB 9.9MB/s 
    100% |████████████████████████████████| 61kB 25.8MB/s 
    100% |████████████████████████████████| 102kB 18.9MB/s 
  Running setup.py bdist_wheel for sshtunnel ... - done
  Stored in directory: /root/.cache/pip/wheels/e1/db/13/8d305bcf8917526b835e6e1658abf3eeb609da87dc8613b834
Successfully built sshtunnel


In [3]:
import datetime as dt
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
!ls gdrive/"My Drive"/"Quanto Singapore"/Kumar/Data/order_book/
kumara_path = 'gdrive/My Drive/Quanto Singapore/Kumar/Data/'
days = [14, 15, 24]
exchanges = ['COINBASE']
crosses = ['ZRX_EUR','ETH_USD']


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
14  15	24


In [4]:

exchange = exchanges[0]
cross = crosses[0]
day = days[0]

#gz_csv = '/order_book/'+str(day)+'/COINBASE_SPOT_ETH_USD.csv.gz'
#data = pd.read_csv(kumara_path+gz_csv, nrows=400000, sep = ';', compression='gzip',error_bad_lines=False)

gz_csv = '/order_book/'+str(day)+'/'+exchange+'_SPOT_'+cross+'.csv.gz'
file_csv = exchange+'_SPOT_'+cross+'.csv'
file = kumara_path + gz_csv

#gz_csv = file_csv + '.gz'
data = pd.read_csv(kumara_path+gz_csv, nrows=400, sep = ';', compression='gzip',error_bad_lines=False)
data.head()



,time_exchange,time_coinapi,update_type,is_buy,entry_px,entry_sx
0,00:00:29.6372524,00:00:29.6372524,SNAPSHOT,0,0.56,22927.34064
1,00:00:29.6372524,00:00:29.6372524,SNAPSHOT,1,0.55,28216.39550
2,00:00:29.6372524,00:00:29.6372524,SNAPSHOT,0,0.57,29933.99058
3,00:00:29.6372524,00:00:29.6372524,SNAPSHOT,1,0.54,19044.26407
4,00:00:29.6372524,00:00:29.6372524,SNAPSHOT,0,0.58,15287.37763


In [0]:
TABLES = {}
TABLES['OrderBook'] = (
    "CREATE TABLE `OrderBook` ("
    "  `id` int(50) NOT NULL AUTO_INCREMENT,"
    "  `time_exchange` varchar(200) NOT NULL,"
    "  `time_coinapi` varchar(200) NOT NULL,"
    "  `update_type` varchar(200) NOT NULL,"
    "  `is_buy` varchar(200) NOT NULL,"
    "  `entry_px` varchar(200) NOT NULL,"
    "  `entry_sx` varchar(200) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

def create_tables(connection, TABLES):
    
  
  
    query = 'show tables;'
    mycursor = connection.cursor(buffered=True)
    for name, ddl in TABLES.items():
        try:
            print("Creating table {}: ".format(name), end='')
            mycursor.execute(ddl)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

In [6]:
import gzip
import shutil
with gzip.open(file, 'rb') as f_in:
    with open(file_csv, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
!ls        

COINBASE_SPOT_ZRX_EUR.csv  gdrive  sample_data


In [0]:
def create_test_table(connection):
    query = 'show tables;'
    cursor = connection.cursor(buffered=True)
    cursor.execute("USE KumarSuppiah$Jarvis") # select the database
    cursor.execute("""CREATE TABLE IF NOT EXISTS TickData(
    TradeID VARCHAR(255) PRIMARY KEY,
    CurrencyPair TEXT)""")
    connection.commit()
    cursor.execute(query)
    for (table_name,) in cursor:
        print(table_name)
        
        
    cursor.execute("""DROP TABLE TickData""")
    connection.commit()
    
    cursor.execute("""DROP TABLE OrderBook""")
    connection.commit()

    cursor.execute(query)
    for (table_name,) in cursor:
        print(table_name)
        
        
def csv_to_mysql(connection, file):
  load_sql = "LOAD DATA LOCAL INFILE '"+file+"' INTO TABLE OrderBook FIELDS TERMINATED BY ';' ENCLOSED BY '\"' IGNORE 1 LINES;"
#  load_sql = "LOAD DATA LOCAL INFILE '/tmp/city.csv' INTO TABLE usermanaged.city\
#        FIELDS TERMINATED BY ',' ENCLOSED BY '\"' IGNORE 1 LINES;"
  cursor = connection.cursor(buffered=True)
  cursor.execute(load_sql) # select the database
  
  
def read_table(connection, table):
  query = "Select * FROM "+table
  cursor = connection.cursor(buffered=True)
  cursor.execute(query) # select the database
  for row in cursor.fetchall():
    print (row)

In [18]:
with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='KumarSuppiah', ssh_password='QuantoTempPass',
    remote_bind_address=('KumarSuppiah.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    connection = mysql.connector.connect(
        user='KumarSuppiah', password='Quanto4all',
        host='127.0.0.1', port=tunnel.local_bind_port,
        database='KumarSuppiah$Jarvis',
    )
    # Do stuff
    create_test_table(connection)
    create_tables(connection, TABLES)
    csv_to_mysql(connection,file_csv)

    read_table(connection, 'OrderBook')
        
    connection.close()

OrderBook
TickData
Creating table OrderBook: OK
(1, '00:00:29.6372524', 'SNAPSHOT', '0', '0.56', '22927.34064\r', '')
(2, '00:00:29.6372524', 'SNAPSHOT', '1', '0.55', '28216.3955\r', '')
(3, '00:00:29.6372524', 'SNAPSHOT', '0', '0.57', '29933.99058\r', '')
(4, '00:00:29.6372524', 'SNAPSHOT', '1', '0.54', '19044.26407\r', '')
(5, '00:00:29.6372524', 'SNAPSHOT', '0', '0.58', '15287.37763\r', '')
(6, '00:00:29.6372524', 'SNAPSHOT', '1', '0.53', '33930.66745\r', '')
(7, '00:00:29.6372524', 'SNAPSHOT', '0', '0.59', '7484.92885\r', '')
(8, '00:00:29.6372524', 'SNAPSHOT', '1', '0.52', '3098.28525\r', '')
(9, '00:00:29.6372524', 'SNAPSHOT', '0', '0.6', '12773.13981\r', '')
(10, '00:00:29.6372524', 'SNAPSHOT', '1', '0.51', '9577.57063\r', '')
(11, '00:00:29.6372524', 'SNAPSHOT', '0', '0.61', '12720.10152\r', '')
(12, '00:00:29.6372524', 'SNAPSHOT', '1', '0.5', '11767.55251\r', '')
(13, '00:00:29.6372524', 'SNAPSHOT', '0', '0.62', '24323.09854\r', '')
(14, '00:00:29.6372524', 'SNAPSHOT', '1', '0

In [0]:
#@title
!pip install sqlalchemy
!pip install psycopg2
from sqlalchemy import create_engine

